In [4]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-a_5xxgg_
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-a_5xxgg_
  Resolved https://github.com/ibm-granite-community/utils to commit 1514191fbbc4605ed4fdfdcb448f2ee41477058f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)

In [6]:
def fewshot_prompt(context, question, game_elements, examples):
    """
    Creates a few-shot prompt for the model, where the model leverages examples to improve accuracy.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question or task for the model to perform
    - game_elements: list, list of core components or features to include in the game
    - examples: list of dicts, each containing 'question', 'context', and 'output' as keys to provide examples

    Returns:
    - str, the formatted few-shot prompt
    """

    # Format game elements like "3x3 grid", "hover effect", etc.
    elements = ", ".join(game_elements)

    # Format the few-shot examples
    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output: {example['output']}
        """
        for i, example in enumerate(examples)
    )

    # Construct the complete prompt
    prompt = f"""
    You are an experienced programmer with 15 years of experience building interactive applications and games using Python and ipywidgets in Jupyter Notebooks.
    Your task is to generate high-quality Python code to enhance the Tic Tac Toe game interface using ipywidgets components.

    Here are some examples of similar tasks you have completed successfully:
    {formatted_examples}

    Now, using these examples as a reference, generate code for the following task:

    Context: {context}
    User Question: {question}
    Include descriptions of elements from the following game features: {elements}
    Ensure that:
    - The code is well-structured and uses consistent styling, coloring, and formatting for all UI elements.
    - Visual feedback is implemented (e.g., hover effect, highlight on click, smooth transitions).
    - The UI components are interactive and intuitive for users.
    - Output only the Python code using ipywidgets (no explanations).
    """
    return prompt

In [7]:
# Function to get answer using few-shot prompting
def get_answer_using_fewshot(context, question, game_elements, examples):
    """
    Generates the response from the model based on a few-shot prompt.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question for the model to answer
    - game_elements: list, list of elements/features to include in the prompt
    - examples: list of dicts with 'question', 'context', and 'output'

    Returns:
    - str, the generated result from the model
    """
    prompt = fewshot_prompt(context, question, game_elements, examples)
    result = model.invoke(prompt)

    return result

In [8]:
examples = [
    {
        "question": "Add a styled header for my Tic Tac Toe game",
        "context": "Gradient Background and Font Styling for Game Title",
        "output": """
import ipywidgets as widgets
from IPython.display import display

header = widgets.HTML(value=\"\"\"
<div style='background: linear-gradient(to right, #0f2027, #203a43, #2c5364); padding: 20px; border-radius: 8px;'>
    <h1 style='color: white; text-align: center; font-family: Arial, sans-serif;'>Tic Tac Toe Game</h1>
</div>
\"\"\")
display(header)
"""
    },
    {
        "question": "Create a 3x3 grid for Tic Tac Toe using ipywidgets",
        "context": "Each cell should be a button in a 3x3 layout. Players take turns by clicking the buttons.",
        "output": """
import ipywidgets as widgets
from IPython.display import display

buttons = [widgets.Button(description='', layout=widgets.Layout(width='60px', height='60px')) for _ in range(9)]
grid = widgets.GridBox(buttons, layout=widgets.Layout(grid_template_columns="repeat(3, 60px)", grid_gap='5px'))
display(grid)
"""
    },
    {
        "question": "Add hover and click highlight to each cell",
        "context": "When user hovers over a button, slightly change color. On click, highlight selected cell with different style.",
        "output": """
# Example styling logic (pseudo)
btn.style.button_color = '#e0e0e0'  # default
btn.add_class('hover-style')  # simulate hover
# On click:
btn.layout.border = '2px solid blue'
btn.layout.background_color = '#d0f0c0'
"""
    },
    {
        "question": "Add a reset button that clears the board and resets the turn",
        "context": "Place a reset button below the grid, when clicked it resets all cells and current player.",
        "output": """
reset_button = widgets.Button(description='Reset Game', button_style='danger')

def reset_game(b):
    for btn in buttons:
        btn.description = ''
    current_player.value = 'X'

reset_button.on_click(reset_game)
display(reset_button)
"""
    }
]

In [9]:
context = "Enhance the UI of a Tic Tac Toe game made with ipywidgets."
question = "Add visual effects such as hover, click highlight, smooth transition, and reset animation to the 3x3 Tic Tac Toe grid."
game_elements = ["3x3 grid", "Player X and O", "hover effect", "highlight on click", "reset animation", "transition style"]

# Generate enhanced Tic Tac Toe UI code using few-shot prompting
result = get_answer_using_fewshot(context, question, game_elements, examples)
print(f"Generated Code:\n{result}")

Generated Code:
```python
import ipywidgets as widgets
from IPython.display import display
import numpy as np

# Initialize game state
board = np.full((3, 3), '')
current_player = 'X'

# Create buttons for the 3x3 grid
buttons = [widgets.Button(description='', layout=widgets.Layout(width='60px', height='60px')) for _ in range(9)]

# Function to update button description and style on click
def on_button_click(change):
    global current_player, board

    if change['new'] == '':
        return

    row, col = divmod(change['owner'].index, 3)
    if board[row][col] == '':
        board[row][col] = current_player
        change['owner'].layout.border = '2px solid black'
        change['owner'].layout.background_color = 'lightgray'
        current_player = 'O' if current_player == 'X' else 'X'

# Apply click handler to each button
for i, btn in enumerate(buttons):
    btn.on_click(lambda b=btn: on_button_click({'new': b.description, 'owner': buttons[i]}) )

# Create a 3x3 grid layout for b

In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# === HEADER ===
header = widgets.HTML(value="""
<div style='background-color: #1e88e5; color: white; padding: 20px; border-radius: 8px;'>
    <h1 style='text-align: center; font-family: Arial, sans-serif;'>Tic Tac Toe Game</h1>
</div>
""")

# === WELCOME MESSAGE ===
welcome = widgets.HTML(value="""
<div style='padding: 10px;'>
    <h3 style='color: #333; text-align: center;'>Player <span style="color: green;" id="current-player">X</span>'s Turn</h3>
</div>
""")

# === STATE ===
current_player = widgets.Label(value="X")
board = [""] * 9
buttons = []

# === EVENT HANDLER ===
def check_winner():
    combos = [(0,1,2), (3,4,5), (6,7,8),  # rows
              (0,3,6), (1,4,7), (2,5,8),  # cols
              (0,4,8), (2,4,6)]           # diagonals
    for a,b,c in combos:
        if board[a] and board[a] == board[b] == board[c]:
            return board[a]
    if all(cell != "" for cell in board):
        return "Draw"
    return None

result = widgets.HTML("")

def create_handler(i):
    def handle_click(b):
        if board[i] == "" and not result.value:
            board[i] = current_player.value
            b.description = current_player.value
            b.layout.background_color = "#c8e6c9" if current_player.value == "X" else "#bbdefb"
            b.layout.border = '2px solid #2e7d32' if current_player.value == "X" else '2px solid #1565c0'

            winner = check_winner()
            if winner:
                result.value = f"<h3 style='color: green;'>🎉 Player {winner} wins!</h3>" if winner != "Draw" else "<h3>It's a draw!</h3>"
            else:
                current_player.value = "O" if current_player.value == "X" else "X"
                welcome.value = f"""
                <div style='padding: 10px;'>
                    <h3 style='color: #333; text-align: center;'>Player <span style="color: green;">{current_player.value}</span>'s Turn</h3>
                </div>
                """
    return handle_click

# === GRID ===
for i in range(9):
    btn = widgets.Button(description="", layout=widgets.Layout(width='60px', height='60px'))
    btn.on_click(create_handler(i))
    buttons.append(btn)

grid = widgets.GridBox(
    buttons,
    layout=widgets.Layout(grid_template_columns="repeat(3, 60px)", grid_gap='5px')
)

# === RESET BUTTON ===
reset_button = widgets.Button(description="Reset Game", button_style='warning')

def reset_game(b):
    global board
    board = [""] * 9
    result.value = ""
    current_player.value = "X"
    welcome.value = """
    <div style='padding: 10px;'>
        <h3 style='color: #333; text-align: center;'>Player <span style="color: green;">X</span>'s Turn</h3>
    </div>
    """
    for btn in buttons:
        btn.description = ""
        btn.layout.background_color = "#f5f5f5"
        btn.layout.border = "1px solid #ccc"

reset_button.on_click(reset_game)

# === LAYOUT ===
main_container = widgets.VBox([
    header,
    welcome,
    grid,
    result,
    reset_button
])

display(main_container)

In [11]:
def refine_output_with_fewshot(previous_output, context, question, game_elements, examples):
    """
    Refines a previously generated Tic Tac Toe UI output by applying additional user-requested changes
    using few-shot prompting technique with IBM Granite.

    Parameters:
    - previous_output: str, the code generated from the initial prompt.
    - context: str, the context for the refinement task.
    - question: str, the user-requested changes to be applied.
    - game_elements: list, list of UI/game elements to consider in the refined output.
    - examples: list of dicts, each containing 'question', 'context', and 'output'

    Returns:
    - str, the refined Python code generated by the model.
    """

    # Format the examples for the prompt
    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output: {example['output']}
        """
        for i, example in enumerate(examples)
    )
    prompt = f"""
    You are an experienced Python developer using ipywidgets to build interactive applications in Jupyter Notebook.
    Your task is to improve an existing Tic Tac Toe UI implementation based on new requirements.
    Below are examples of similar UI enhancements:
    {formatted_examples}

Now, I have a basic Tic Tac Toe game interface created from an initial prompt. The game currently uses ipywidgets in Jupyter Notebook to render a 3x3 grid and manage player turns.

    Context:
    {context}

    User Question:
    {question}
    """
    # Use the model to generate the refined code
    result = model.invoke(prompt)
    return result

In [14]:
# Example usage
context = "Make the Tic Tac Toe game responsive and more interactive"
question = "Add player score tracking, highlight the winning cells, and animate the board reset"
# Generate and display the UI code for the landing page
ref_result = refine_output_with_fewshot(
    previous_output=result,
    context=context,
    question=question,
    game_elements=game_elements,
    examples=examples
)

print("Generated Code:\n", ref_result)

Generated Code:
 To address your requirements, we'll enhance the existing Tic Tac Toe UI by adding player score tracking, highlighting winning cells, and animating the board reset. Here's a step-by-step implementation:

1. **Player Score Tracking:** We'll add two score displays for 'X' and 'O' players.
2. **Highlighting Winning Cells:** We'll add a function to detect wins and highlight the winning cells.
3. **Animating Board Reset:** We'll use a simple animation effect when resetting the board.

Here's the updated code:

```python
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# Initialize game state
current_player = 'X'
board = ['' for _ in range(9)]
scores = {'X': 0, 'O': 0}

# Create styled header
header = widgets.HTML(value="""
<div style='background: linear-gradient(to right, #0f2027, #203a43, #2c5364); padding: 20px; border-radius: 8px;'>
    <h1 style='color: white; text-align: center; font-family: Arial, sans-serif;'>Tic Tac Toe Game

In [15]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# === HEADER ===
header = widgets.HTML(value="""
<div style='background-color: #333; color: white; padding: 20px; border-radius: 8px;'>
    <h1 style='text-align: center; font-family: Arial, sans-serif;'>Tic Tac Toe</h1>
</div>
""")

# === WELCOME MESSAGE ===
welcome = widgets.HTML(value="""
<div style='padding: 10px; border-radius: 8px;'>
    <h2 style='color: white; text-align: center;'>Player <span style="color: #90caf9;">X</span>'s Turn</h2>
</div>
""")

# === SCORE TRACKER ===
score = {"X": 0, "O": 0}
score_board = widgets.HTML(value=f"""
<div style='color: white; text-align: center; margin-bottom: 10px;'>
    <h3>Score — X: {score["X"]} | O: {score["O"]}</h3>
</div>
""")

# === STATE ===
current_player = widgets.Label(value="X")
board = [""] * 9
buttons = []

# === GAME LOGIC ===
def check_winner():
    win_combos = [(0,1,2), (3,4,5), (6,7,8),
                  (0,3,6), (1,4,7), (2,5,8),
                  (0,4,8), (2,4,6)]
    for a,b,c in win_combos:
        if board[a] and board[a] == board[b] == board[c]:
            return board[a], [a, b, c]
    if all(cell != "" for cell in board):
        return "Draw", []
    return None, []

result = widgets.HTML("")

def update_score(winner):
    if winner in score:
        score[winner] += 1
    score_board.value = f"""
    <div style='color: white; text-align: center; margin-bottom: 10px;'>
        <h3>Score — X: {score["X"]} | O: {score["O"]}</h3>
    </div>
    """

def create_handler(i):
    def handle_click(b):
        if board[i] == "" and not result.value:
            board[i] = current_player.value
            b.description = current_player.value
            b.layout.background_color = "#c8e6c9" if current_player.value == "X" else "#bbdefb"
            b.layout.border = "2px solid #2e7d32" if current_player.value == "X" else "2px solid #1565c0"
            winner, win_cells = check_winner()
            if winner:
                if winner != "Draw":
                    result.value = f"<h3 style='color: #81c784;'>🎉 Player {winner} wins!</h3>"
                    for idx in win_cells:
                        buttons[idx].layout.background_color = "#ffeb3b"
                    update_score(winner)
                else:
                    result.value = "<h3 style='color: #ffb300;'>It's a draw!</h3>"
            else:
                current_player.value = "O" if current_player.value == "X" else "X"
                welcome.value = f"""
                <div style='padding: 10px; border-radius: 8px;'>
                    <h2 style='color: white; text-align: center;'>Player <span style="color: #90caf9;">{current_player.value}</span>'s Turn</h2>
                </div>
                """
    return handle_click

# === CREATE BUTTONS WITH HOVER EFFECT ===
for i in range(9):
    btn = widgets.Button(
        description="",
        layout=widgets.Layout(width='70px', height='70px', transition='transform 0.2s'),
        style=widgets.ButtonStyle(button_color="#eeeeee")
    )
    btn.on_click(create_handler(i))
    buttons.append(btn)

grid = widgets.GridBox(
    buttons,
    layout=widgets.Layout(grid_template_columns="repeat(3, 70px)", grid_gap='10px', justify_content='center')
)

# === RESET BUTTON ===
reset_btn = widgets.Button(description="Reset Game", button_style='danger')

def reset_game(b):
    global board
    board = [""] * 9
    current_player.value = "X"
    result.value = ""
    welcome.value = """
    <div style='padding: 10px; border-radius: 8px;'>
        <h2 style='color: white; text-align: center;'>Player <span style="color: #90caf9;">X</span>'s Turn</h2>
    </div>
    """
    for btn in buttons:
        btn.description = ""
        btn.layout.background_color = "#eeeeee"
        btn.layout.border = "1px solid #ccc"

reset_btn.on_click(reset_game)

# === CONTAINER ===
main_container = widgets.VBox([
    header,
    welcome,
    score_board,
    grid,
    result,
    reset_btn
])

# === BACKGROUND WRAPPER (Optional) ===
wrapper = widgets.HTML(value="""
<style>
    body {
        background-color: #212121;
    }
</style>
""")

display(wrapper, main_container)

HTML(value='\n<style>\n    body {\n        background-color: #212121;\n    }\n</style>\n')